# Transfer learning with Huggingface using CodeFlare

In this notebook you will learn how to leverage the **[huggingface](https://huggingface.co/)** support in ray ecosystem to carry out a text classification task using transfer learning. We will be referencing the example **[here](https://huggingface.co/docs/transformers/tasks/sequence_classification)**

The example carries out a text classification task on **[imdb dataset](https://huggingface.co/datasets/imdb)** and tries to classify the movie reviews as positive or negative. Huggingface library provides an easy way to build a model and the dataset to carry out this classification task. In this case we will be using **distilbert-base-uncased** model which is a **BERT** based model.

Huggingface has a **[built in support for ray ecosystem](https://docs.ray.io/en/releases-1.13.0/_modules/ray/ml/train/integrations/huggingface/huggingface_trainer.html)** which allows the huggingface trainer to scale on CodeFlare and can scale the training as we add additional gpus and can run distributed training across multiple GPUs that will help scale out the training.


### Getting all the requirements in place

In [12]:
# Import pieces from codeflare-sdk
from codeflare_sdk.cluster.cluster import Cluster, ClusterConfiguration
from codeflare_sdk.cluster.auth import TokenAuthentication

In [ ]:
# Create authentication object for oc user permissions and login
auth = TokenAuthentication(
    token = "XXXX",
    server = "XXXX",
    skip_tls = True
)
auth.login()

Here, we want to define our cluster by specifying the resources we require for our batch workload. Below, we define our cluster object (which generates a corresponding AppWrapper).

In [13]:
# Create our cluster and submit appwrapper
cluster = Cluster(ClusterConfiguration(name='hfgputest', min_worker=1, max_worker=1, min_cpus=8, max_cpus=8, min_memory=16, max_memory=16, gpu=4, instascale=True, machine_types=["m5.xlarge", "p3.8xlarge"]))

Written to: hfgputest.yaml


Next, we want to bring our cluster up, so we call the `up()` function below to submit our cluster AppWrapper yaml onto the MCAD queue, and begin the process of obtaining our resource cluster.

In [ ]:
cluster.up()

Now, we want to check on the initial status of our resource cluster, then wait until it is finally ready for use.

In [17]:
cluster.status()

╭─────────────────────────╮
│   🚀 List of CodeFlare  │
│   clusters in queue🚀   │
│ +-----------+---------+ │
│ | Name      | Status  | │
│ +===========+=========+ │
│ | hfgputest | pending | │
│ |           |         | │
│ +-----------+---------+ │
╰─────────────────────────╯

(False, <CodeFlareClusterStatus.QUEUED: 2>)

In [ ]:
cluster.wait_ready()

In [ ]:
cluster.status()

Let's quickly verify that the specs of the cluster are as expected.

In [18]:
cluster.details()

                  🚀 List of CodeFlare clusters 🚀                  
                                                                    
 ╭────────────────────────────────────────────────────────────────╮ 
 │   Owner                                                        │ 
 │   hfgputest                                        Active ✅   │ 
 │                                                                │ 
 │   URI: ray://hfgputest-head-svc.default.svc:10001              │ 
 │                                                                │ 
 │   ]8;id=552692;ray-dashboard-hfgputest-default.apps.prepfullinstall.psap.aws.rhperfscale.org\Dashboard🔗]8;;\                                                  │ 
 │                                                                │ 
 │                      Cluster Resources                         │ 
 │   ╭─ Workers ──╮  ╭───────── Worker specs(each) ─────────╮     │ 
 │   │  Min  Max  │  │  Memory      CPU         GPU         │     │ 
 │   │            │  │                                      │     │ 
 │   │  1    1    │  │  16G~16G     8           4           │     │ 
 │   │            │  │                                      │     │ 
 │   ╰────────────╯  ╰──────────────────────────────────────╯     │ 
 ╰────────────────────────────────────────────────────────────────╯

<RayClusterStatus.READY: 'ready'>

In [19]:
ray_cluster_uri = cluster.cluster_uri()

**NOTE**: Now we have our resource cluster with the desired GPUs, so we can interact with it to train the HuggingFace model.

In [20]:
#before proceeding make sure the cluster exists and the uri is not empty
assert ray_cluster_uri, "Ray cluster needs to be started and set before proceeding"

import ray
from ray.air.config import ScalingConfig

# reset the ray context in case there's already one. 
ray.shutdown()
# establish connection to ray cluster

#install additionall libraries that will be required for this training
runtime_env = {"pip": ["transformers", "datasets", "evaluate"]}

ray.init(address=f'{ray_cluster_uri}', runtime_env=runtime_env)

print("Ray cluster is up and running: ", ray.is_initialized())

Ray cluster is up and running:  True


**NOTE** : in this case since we are running a task for which we need additional pip packages. we can install those by passing them in the `runtime_env` variable

### Transfer learning code from huggingface

We are using the code based on the example **[here](https://huggingface.co/docs/transformers/tasks/sequence_classification)** . 

In [21]:
@ray.remote
def train_fn():
    from datasets import load_dataset
    import transformers
    from transformers import AutoTokenizer, TrainingArguments
    from transformers import AutoModelForSequenceClassification
    import numpy as np
    from datasets import load_metric
    import ray
    from ray import tune
    from ray.train.huggingface import HuggingFaceTrainer

    dataset = load_dataset("imdb")
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    #using a fraction of dataset but you can run with the full dataset
    small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
    small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

    print(f"len of train {small_train_dataset} and test {small_eval_dataset}")

    ray_train_ds = ray.data.from_huggingface(small_train_dataset)
    ray_evaluation_ds = ray.data.from_huggingface(small_eval_dataset)

    def compute_metrics(eval_pred):
        metric = load_metric("accuracy")
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    def trainer_init_per_worker(train_dataset, eval_dataset, **config):
        model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

        training_args = TrainingArguments("/tmp/hf_imdb/test", eval_steps=1, disable_tqdm=True, 
                                          num_train_epochs=1, skip_memory_metrics=True,
                                          learning_rate=2e-5,
                                          per_device_train_batch_size=16,
                                          per_device_eval_batch_size=16,                                
                                          weight_decay=0.01,)
        return transformers.Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            compute_metrics=compute_metrics
        )

    scaling_config = ScalingConfig(num_workers=4, use_gpu=True) #num workers is the number of gpus

    # we are using the ray native HuggingFaceTrainer, but you can swap out to use non ray Huggingface Trainer. Both have the same method signature. 
    # the ray native HFTrainer has built in support for scaling to multiple GPUs
    trainer = HuggingFaceTrainer(
        trainer_init_per_worker=trainer_init_per_worker,
        scaling_config=scaling_config,
        datasets={"train": ray_train_ds, "evaluation": ray_evaluation_ds},
    )
    result = trainer.fit()


**NOTE:** This code will produce a lot of output and will run for **approximately 2 minutes.** As a part of execution it will download the `imdb` dataset, `distilbert-base-uncased` model and then will start transfer learning task for training the model with this dataset. 

In [22]:
#call the above cell as a remote ray function
ray.get(train_fn.remote())

(train_fn pid=250) Downloading and preparing dataset imdb/plain_text to /home/ray/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1...


Generating train split: 100%|█████████▉| 24881/25000 [00:05<00:00, 7674.07 examples/s]
                                                                                      
Generating unsupervised split:  99%|█████████▉| 49641/50000 [00:10<00:00, 7818.59 examples/s]


(train_fn pid=250) Dataset imdb downloaded and prepared to /home/ray/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 696.30it/s]                                               
Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 32.1kB/s]
Downloading: 100%|██████████| 483/483 [00:00<00:00, 600kB/s]
Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 4.80MB/s]
Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]
Downloading: 100%|██████████| 466k/466k [00:00<00:00, 7.88MB/s]
 98%|█████████▊| 49/50 [00:31<00:00,  1.54ba/s]


(train_fn pid=250) len of train Dataset({
(train_fn pid=250)     features: ['text', 'label', 'input_ids', 'attention_mask'],
(train_fn pid=250)     num_rows: 100
(train_fn pid=250) }) and test Dataset({
(train_fn pid=250)     features: ['text', 'label', 'input_ids', 'attention_mask'],
(train_fn pid=250)     num_rows: 100
(train_fn pid=250) })


...

(BaseWorkerMixin pid=182, ip=10.129.66.16) 
(BaseWorkerMixin pid=182, ip=10.129.66.16) 
(BaseWorkerMixin pid=182, ip=10.129.66.16) Training completed. Do not forget to share your model on huggingface.co/models =)
(BaseWorkerMixin pid=182, ip=10.129.66.16) 
(BaseWorkerMixin pid=182, ip=10.129.66.16) 


(train_fn pid=250) == Status ==
(train_fn pid=250) Current time: 2022-11-04 07:58:13 (running for 00:02:19.36)
(train_fn pid=250) Memory usage on this node: 16.0/240.1 GiB
(train_fn pid=250) Using FIFO scheduling algorithm.
(train_fn pid=250) Resources requested: 5.0/10 CPUs, 4.0/4 GPUs, 0.0/22.35 GiB heap, 0.0/6.59 GiB objects
(train_fn pid=250) Result logdir: /home/ray/ray_results/HuggingFaceTrainer_2022-11-04_07-55-53
(train_fn pid=250) Number of trials: 1/1 (1 RUNNING)
(train_fn pid=250) +--------------------------------+----------+------------------+--------+------------------+-----------------+----------------------------+--------------------------+
(train_fn pid=250) | Trial name                     | status   | loc              |   iter |   total time (s) |   train_runtime |   train_samples_per_second |   train_steps_per_second |
(train_fn pid=250) |--------------------------------+----------+------------------+--------+------------------+-----------------+---------------------

(train_fn pid=250) 2022-11-04 07:58:13,248	WARNING util.py:214 -- The `process_trial_save` operation took 9.709 s, which may be a performance bottleneck.
(train_fn pid=250) 2022-11-04 07:58:13,248	WARNING trial_runner.py:856 -- Consider turning off forced head-worker trial checkpoint syncs by setting sync_on_checkpoint=False. Note that this may result in faulty trial restoration if a failure occurs while the checkpoint is being synced from the worker to the head node.


(train_fn pid=250) Result for HuggingFaceTrainer_c7d60_00000:
(train_fn pid=250)   _time_this_iter_s: 118.07144260406494
(train_fn pid=250)   _timestamp: 1667573883
(train_fn pid=250)   _training_iteration: 1
(train_fn pid=250)   date: 2022-11-04_07-58-03
(train_fn pid=250)   done: true
(train_fn pid=250)   epoch: 1.0
(train_fn pid=250)   experiment_id: 7bc6ab25d0414fcbb589bcb5d0f29b99
(train_fn pid=250)   experiment_tag: '0'
(train_fn pid=250)   hostname: hfgputest-worker-small-group-hfgputest-q4758
(train_fn pid=250)   iterations_since_restore: 1
(train_fn pid=250)   node_ip: 10.129.66.16
(train_fn pid=250)   pid: 146
(train_fn pid=250)   should_checkpoint: true
(train_fn pid=250)   step: 391
(train_fn pid=250)   time_since_restore: 124.55581378936768
(train_fn pid=250)   time_this_iter_s: 124.55581378936768
(train_fn pid=250)   time_total_s: 124.55581378936768
(train_fn pid=250)   timestamp: 1667573883
(train_fn pid=250)   timesteps_since_restore: 0
(train_fn pid=250)   train_loss: 

(train_fn pid=250) 2022-11-04 07:58:16,286	WARNING util.py:214 -- The `process_trial_save` operation took 2.161 s, which may be a performance bottleneck.
(train_fn pid=250) 2022-11-04 07:58:16,398	INFO tune.py:747 -- Total run time: 142.70 seconds (142.40 seconds for the tuning loop).


Finally, we bring our resource cluster down and release/terminate the associated resources, bringing everything back to the way it was before our cluster was brought up.

In [ ]:
cluster.down()

In [ ]:
auth.logout()

## Conclusion
As shown in the above example, you can easily run your Huggingface transfer learning tasks easily and natively on CodeFlare. You can scale them from 1 to n GPUs without requiring you to make any significant code changes and leveraging the native Huggingface trainer. 

Also refer to additional notebooks that showcase other use cases
In our next notebook [./02_codeflare_workflows_encoding.ipynb ] shows an sklearn example and how you can leverage workflows to run experiment pipelines and explore multiple pipelines in parallel on CodeFlare cluster. 
